# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [3]:
# Get the datasets
!wget http://huang.eng.unt.edu/CSCE-5218/test.dat
!wget http://huang.eng.unt.edu/CSCE-5218/train.dat


--2024-02-17 17:01:59--  http://huang.eng.unt.edu/CSCE-5218/test.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2844 (2.8K)
Saving to: ‘test.dat’

test.dat            100%[===================>]   2.78K  --.-KB/s    in 0s      

2024-02-17 17:02:00 (273 MB/s) - ‘test.dat’ saved [2844/2844]

--2024-02-17 17:02:00--  http://huang.eng.unt.edu/CSCE-5218/train.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11244 (11K)
Saving to: ‘train.dat’

train.dat           100%[===================>]  10.98K  --.-KB/s    in 0s      

2024-02-17 17:02:00 (131 MB/s) - ‘train.dat’ saved [11244/11244]



In [1]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [9]:
import math
import itertools
import re
import numpy as np


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    #TODO: Return dot product of array 1 and array 2
    dot = 0
    for i in range(len(array1)):
        dot += array1[i] * array2[i]
    return dot 


def sigmoid(x):
    #TODO: Return outpout of sigmoid function on x
    sigmoid = 1 / (1+math.e** (-x))
    return sigmoid

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    #TODO: return the output of the model
    in_values = dot_product(weights, instance)
    output = sigmoid(in_values)
    return output

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    #TODO: return the prediction of the model
    in_values = dot_product(weights, instance)
    output = sigmoid(in_values)
    return 1 if output >= 0.5 else 0


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    #Initializing weights
    weights = [0] * (len(instances[0]))
    # print(len(weights))

    for _ in range(epochs):
        for instance in instances:
            # print(len(instance))
            #Forward Propagation
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output
            #Implementing Backpropagation using loss function and chain rule
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [13]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### Answer
To answer this question, we need to understand the output and predict functions clearly. Output function produces the activations of the dot product of weights and instances on the other hand predict function does the same but it does an extra step, that is it marks the instance as 0 (if sigmoid op < 0.5) or 1 (if sigmoid op >= 0.5) based on the sigmoid activation values. But we need the raw activation values to calculate the error of the model which can be used for backpropagation that helps the model to learn its mistakes and adjust the weights accordingly.



### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 

#### Code
```
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      size =  round(len(instances_tr)*tr_size/100)
      pre_instances = instances_tr[0:size]
      weights = train_perceptron(pre_instances, lr, epochs)
      accuracy = get_accuracy(weights, instances_te)
      print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
            f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")
```
#### OUTPUT
```
#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 74.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 77.0
#tr: 100, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 92.0
#tr: 200, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 200, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 80.0
#tr: 200, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 97.0
#tr: 200, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 300, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 71.0
#tr: 300, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 92.0
#tr: 300, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 99.0
#tr: 300, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 400, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 80.0
#tr: 400, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 96.0
#tr: 400, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 100.0
#tr: 20, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 74.0
#tr: 20, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 85.0
#tr: 40, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 80.0
#tr: 100, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 70.0
#tr: 100, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 92.0
#tr: 100, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 98.0
#tr: 200, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 200, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 83.0
#tr: 200, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 96.0
#tr: 200, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 200, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 71.0
#tr: 300, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 92.0
#tr: 300, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 98.0
#tr: 300, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 80.0
#tr: 400, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 96.0
#tr: 400, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 100.0
#tr: 20, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 74.0
#tr: 20, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 85.0
#tr: 20, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 91.0
#tr: 20, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 91.0
#tr: 40, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 78.0
#tr: 40, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 90.0
#tr: 40, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 97.0
#tr: 100, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 80.0
#tr: 100, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 94.0
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 98.0
#tr: 100, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 100, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 200, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 98.0
#tr: 200, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 200, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 200, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 200, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 99.0
#tr: 300, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 300, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
#tr: 400, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
```

In [16]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      size =  round(len(instances_tr)*tr_size/100)
      pre_instances = instances_tr[0:size]
      weights = train_perceptron(pre_instances, lr, epochs)
      accuracy = get_accuracy(weights, instances_te)
      print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
            f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 74.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, lea

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)

#### Answer A
It is not always necessary to train the model with all the training dataset, it depends on the complexity of the model and the amount of data we have in the dataset. This has clearly been proven from above code block. But we may need to train the model for longer with less data to achieve better results. We were able to achive 100% accuracy with only 200 data points by adjusting learning rate. So, we not always need to train the model with the entire dataset 

#### Answer B
If we use the same set of parameters we will always get the same results unless we use randon intializations (which can be solved using a common seed value). But this is only true if the size of the dataset is same, but if the size of the dataset changes this may affect the hyperparameters and we may see some different results compared to the initial run. If the data is good then we may see better results or bad results if the second part of the dataset is not good. Also if the hyperparameters are not chosen correctly for the second run can result in bad results. It is always a good idea to play with different values to find the best hyperparameters.

#### Answer C
In the model that we used in this task it is not necessary to introduce new parameters, since we were able to achieve 100% accuracy just by tuning the size of dataset, learning rate and epochs. But it may be tru in some cases where we deal with complex problems or reasearch activities. 

#### Answer D
It totally depends on the observations made while testing. If the hyperparameters are selected correctly and the model is doing well and reaches 100% accuracy at 100 epochs then it is not necessary to train the model for 500 epoochs, since it has already reached the saturation point and training any further wont do any good. So, to find this saturation point we should experiment with different epoch values. In reality, instead of depending on number of epochs it is good to stop the training based on a particular metric, since we dont know how long we need to train the model to reach that point.